In [37]:
from typing import Any

from common.inputreader import InputReader, PuzzleWrapper

puzzle = PuzzleWrapper(year=2024, day=int("15"))

puzzle.header()
# example = get_code_block(puzzle, 5)

# Warehouse Woes

[Open Website](https://adventofcode.com/2024/day/15)

In [38]:
from common.matrix import Matrix, MatrixNavigator


# helper functions
def domain_from_input(input: InputReader) -> (Matrix, list):
    lines = input.lines_as_str()

    matrix = []
    instructions = []

    # building map
    building_map = True
    for line in lines:
        if building_map:
            # split on characters
            matrix.append(list(line))

            if len(line) == 0:
                building_map = False
        else:
            for next in line:
                instructions.append(next)

    return Matrix(matrix), instructions


matrix, instructions = domain_from_input(puzzle.example(0))
matrix.print()

##########
#..O..O.O#
#......O.#
#.OO..O.O#
#..O@..O.#
#O#..O...#
#O..O..O.#
#.OO.O.OO#
#....O...#
##########



In [39]:
from common.matrix import Direction

# test case (part 1)
instructions_map = {
    ">": Direction.RIGHT,
    "<": Direction.LEFT,
    "^": Direction.UP,
    "v": Direction.DOWN
}


def part_1(reader: InputReader, debug: bool) -> int:
    matrix, instructions = domain_from_input(reader)

    if debug:
        matrix.print()

    # find start
    pointer = None
    for x, y, value in matrix:
        if value == "@":
            pointer = MatrixNavigator(matrix, x, y)
            break

    # run through instructions
    for instruction in instructions:
        direction = instructions_map[instruction]
        _, value = pointer.peek_value(direction)
        if value == ".":  # open
            pointer.set_value(".")
            pointer.move(direction)
            pointer.set_value("@")
        elif value == "O":  # wall
            # copy pointers
            candidate = pointer.copy()
            candidate.move(direction)
            scout = candidate.copy()
            done = False
            while not done:
                scout.move(direction)
                value = scout.get_value()
                if value == ".":
                    scout.set_value("O")
                    pointer.set_value(".")
                    pointer.move(direction)
                    pointer.set_value("@")
                    done = True
                if value == "#":  # hit wall, stop
                    done = True

    total = 0
    for x, y, value in matrix:
        if value == "O":
            total += y * 100 + x

    return total


result = part_1(puzzle.get_code_block(1), True)
display(result)
assert result == 2028

result = part_1(puzzle.get_code_block(0), True)
display(result)
assert result == 10092

########
#..O.O.#
##@.O..#
#...O..#
#.#.O..#
#...O..#
#......#
########



2028

##########
#..O..O.O#
#......O.#
#.OO..O.O#
#..O@..O.#
#O#..O...#
#O..O..O.#
#.OO.O.OO#
#....O...#
##########



10092

In [40]:
# real case (part 1)
result = part_1(puzzle.input(), False)
display(result)
assert result == 1442192

1442192

In [41]:
# test case (part 2)
def convert_matrix(matrix: Matrix) -> Matrix:
    new_matrix = []
    row = []
    last_y = -1
    for _, y, value in matrix:
        # if new row
        if last_y != y:
            if len(row) > 0:
                new_matrix.append(row)
            row = []
        last_y = y

        if value == "#":
            row.append("#")
            row.append("#")
        elif value == ".":
            row.append(".")
            row.append(".")
        elif value == "@":
            row.append("@")
            row.append(".")
        elif value == "O":
            row.append("[")
            row.append("]")

    if len(row) > 0:
        new_matrix.append(row)

    return Matrix(new_matrix)


def find_box(boxes: list, position):
    x, y = position
    for box in boxes:
        if (box[0] == x or box[0] + 1 == x) and box[1] == y:
            return box
    return None


def can_box_move_horizontal(matrix: Matrix, box: list, direction: Direction) -> list:
    boxes = find_boxes(matrix)
    x, y = box
    pointer = MatrixNavigator(matrix, x, y)
    effected_boxes = [box]
    while True:
        pointer.move(direction)
        value = pointer.get_value()
        if value == "#":
            return []  # hit wall, stop
        elif value == ".":
            return effected_boxes
        else:
            box = find_box(boxes, pointer.current_position)
            effected_boxes.append(box)


def can_box_move_vertical(matrix: Matrix, box: list, direction: Direction) -> list:
    boxes = find_boxes(matrix)
    x, y = box
    effected_boxes = [box]

    # create two pointers
    left_pointer = MatrixNavigator(matrix, x, y)
    right_pointer = MatrixNavigator(matrix, x + 1, y)
    left_pointer.move(direction)
    right_pointer.move(direction)
    left_value = left_pointer.get_value()
    right_value = right_pointer.get_value()

    if left_value == "#" or right_value == "#":
        return []
    elif left_value == "." and right_value == ".":  # open
        return effected_boxes
    else:
        if left_value == "[" or left_value == "]":
            left_box = find_box(boxes, left_pointer.current_position)
            effected_boxes.append(left_box)
            new_boxes_left = can_box_move_vertical(matrix, left_box, direction)
            if len(new_boxes_left) == 0:
                return []
            else:
                for next_box in new_boxes_left:
                    effected_boxes.append(next_box)

        if right_value == "[":
            right_box = find_box(boxes, right_pointer.current_position)
            effected_boxes.append(right_box)
            new_boxes_right = can_box_move_vertical(matrix, right_box, direction)
            if len(new_boxes_right) == 0:
                return []
            else:
                for next_box in new_boxes_right:
                    effected_boxes.append(next_box)

        return list(set(effected_boxes))


def find_boxes(matrix: Matrix) -> list:
    boxes = []
    for x, y, value in matrix:
        if value == "[":
            boxes.append((x, y))
    return boxes


def part_2(reader: InputReader, debug: bool) -> int:
    matrix, instructions = domain_from_input(reader)
    matrix = convert_matrix(matrix)

    # find start
    pointer = None
    for x, y, value in matrix:
        if value == "@":
            pointer = MatrixNavigator(matrix, x, y)
            break

    # clear output.txt
    with open("output.txt", "w") as f:
        f.write("")

    def log_state(matrix: Matrix, instruction: str, count: int):
        with open("output.txt", "a") as f:
            for line in matrix.get_lines():
                f.write(f'{"".join(line)}\n')
            f.write(f'Instruction: {instruction} - {count}\n')
            f.write("\n")

    # run through instructions
    count = 0
    for instruction in instructions:
        if debug:
            log_state(matrix, instruction, count)
        count += 1

        direction = instructions_map[instruction]
        _, value = pointer.peek_value(direction)
        if value == ".":  # open
            pointer.set_value(".")
            pointer.move(direction)
            pointer.set_value("@")
        elif value == "[" or value == "]":  # box
            # Create new point and fix box
            candidate = pointer.copy()
            candidate.move(direction)
            box = find_box(find_boxes(matrix), candidate.current_position)
            if direction == Direction.LEFT or direction == Direction.RIGHT:
                effected_boxes = can_box_move_horizontal(matrix, box, direction)
            else:
                effected_boxes = can_box_move_vertical(matrix, box, direction)
            effected_boxes = list(set(effected_boxes))

            # move all the effected boxes and redraw the matrix
            for box in effected_boxes:
                box_x, box_y = box
                box_pointer = MatrixNavigator(matrix, box_x, box_y)
                box_pointer.set_value(".")
                box_pointer.move(direction.RIGHT)
                box_pointer.set_value(".")

            for box in effected_boxes:
                box_x, box_y = box
                box_pointer = MatrixNavigator(matrix, box_x, box_y)
                box_pointer.move(direction)
                box_pointer.set_value("[")
                box_pointer.move(direction.RIGHT)
                box_pointer.set_value("]")

            if len(effected_boxes) > 0:
                pointer.set_value(".")
                pointer.move(direction)
                pointer.set_value("@")

    if debug:
        log_state(matrix, "*", count)
        matrix.print()

    total = 0
    for x, y, value in matrix:
        if value == "[":
            total += y * 100 + x

    return total


result = part_2(puzzle.get_code_block(5), True)
print(result)
assert result == 618

result = part_2(puzzle.get_code_block(0), True)
print(result)
assert result == 9021

##############
##...[].##..##
##...@.[]...##
##....[]....##
##..........##
##..........##
##############
618
####################
##[].......[].[][]##
##[]...........[].##
##[]........[][][]##
##[]......[]....[]##
##..##......[]....##
##..[]............##
##..@......[].[][]##
##......[][]..[]..##
####################
9021


In [42]:
# real case (part 2)
result = part_2(puzzle.input(), False)
display(result)
assert result == 1448458

1448458

In [43]:
# print easters eggs
puzzle.print_easter_eggs()

## Easter Eggs

<span title="Wesnoth players might solve their Warehouse Woes with a Warehouse Wose!">amok</span> (Wesnoth players might solve their Warehouse Woes with a Warehouse Wose!)